# A pytorch ML adapter demo

#### Preamble
Demo of a relatively small pytorch model.
This notebook uses the [ml_adapter_torch](../../env/ml_adapter_torch) dependencies.
To start: 
```
bin/jupyter_notebook env/ml_adapter_torch ml_adapter/torch_autoencoder
```

In [1]:
!python --version

Python 3.11.9


In [2]:
# the sdk profile used to connect
PROFILE='_default_'
LOG_LEVEL='INFO'
MODEL_NAME='autoencoderV1'
MODEL_VERSION='1.0.0'

In [3]:
# setup INFO logging to see http requests made.
import logging
logging.basicConfig(
    format='%(asctime)s %(levelname)-8s %(message)s',
    level=LOG_LEVEL,
    datefmt='%Y-%m-%d %H:%M:%S'
)

## Pytorch example

A simple auto-encoder pytorch model.

In [4]:
from autoencoder import AutoEncoder

In [5]:
# we saved our model class in a `autoencoder.py` file
from IPython.display import Code, Markdown
display(Code(filename='autoencoder.py'))
from autoencoder import AutoEncoder

import torch

class AutoEncoder(torch.nn.Module):
    def __init__(self):
        super().__init__()
        """
        Create a simple AutoEncoder
        """
        # Use an AutoEncoder and try to reconstruct both signals (the 20 samples back)
        self.encoder = torch.nn.Sequential(
            torch.nn.Linear(20, 10),
            torch.nn.ReLU(),
            torch.nn.Linear(10, 5),
        )

        self.decoder = torch.nn.Sequential(
            torch.nn.Linear(5, 10),
            torch.nn.ReLU(),
            torch.nn.Linear(10, 20),
        )

    def forward(self, x):
        encoded = self.encoder(x)
        decoded = self.decoder(encoded)
        return decoded

In [6]:
# some pretrained weights
weights_path = 'AutoEncoderWeights.pth'

In [7]:
import torch
model = AutoEncoder()
model.load_state_dict(torch.load(weights_path))
model.eval()

AutoEncoder(
  (encoder): Sequential(
    (0): Linear(in_features=20, out_features=10, bias=True)
    (1): ReLU()
    (2): Linear(in_features=10, out_features=5, bias=True)
  )
  (decoder): Sequential(
    (0): Linear(in_features=5, out_features=10, bias=True)
    (1): ReLU()
    (2): Linear(in_features=10, out_features=20, bias=True)
  )
)

In [8]:
x_data = torch.randn(20, dtype=torch.float32)
x_data

tensor([ 0.6440,  0.7529,  1.1046,  1.1565,  0.5734,  0.0943, -0.0062, -0.3886,
         0.1544,  0.2429,  1.3781, -1.0619,  1.0171, -0.8512, -0.3010, -0.4041,
         0.7137, -0.4662,  0.1278,  1.2790])

In [9]:
preds = model(x_data)
preds

tensor([0.2374, 0.2425, 0.2575, 0.2413, 0.2440, 0.2275, 0.2310, 0.2287, 0.2211,
        0.2760, 0.2447, 0.2860, 0.2459, 0.2123, 0.2231, 0.2105, 0.2465, 0.1848,
        0.2116, 0.2709], grad_fn=<ViewBackward0>)

## The adapter
The `V1TorchAdapter` from the `ml_adapter.torch` module wraps our model in a script that can be used as a waylay webscript or plug.

In [13]:
# choose a local archive location
ARCHIVE_LOC = 'autoencoder-pytorch'
# make sure its empty
!rm -fr autoencoder-pytorch

In [16]:
from ml_adapter.torch import V1TorchAdapter
# create an ML adapter to wrap our model
# by using a `weights.pt` postfix we are storing only the weights when serializing the model
MODEL_PATH='model-weights.pt'
adapter = V1TorchAdapter(model=model, model_path='model-weights.pt', location=ARCHIVE_LOC)

In [17]:
# the adapter exposes your model with a REST-compatible interface
result = await adapter.call({"instances": [ x_data.tolist() ]}) 
result

{'predictions': [[0.23736661672592163,
   0.24252289533615112,
   0.2575299143791199,
   0.24128106236457825,
   0.24399690330028534,
   0.22746336460113525,
   0.2309776097536087,
   0.2286645472049713,
   0.22113662958145142,
   0.27599504590034485,
   0.24474786221981049,
   0.28604230284690857,
   0.24586451053619385,
   0.21225886046886444,
   0.223057359457016,
   0.21050433814525604,
   0.2464890033006668,
   0.18476180732250214,
   0.21161124110221863,
   0.27093297243118286]]}

In [18]:
# because we store only weights, the adapter archive needs to now about autoencode model class:
await adapter.add_script('autoencoder.py')

autoencoder.py <ml_adapter.base.assets.python.PythonScriptAsset>

### Creating the plug
Tell the adapter to configure itself as a plug: this generates a number of _assets_ that will be uploaded and define the plug behaviour
* a `plug.json` _manifest_ file that defines the name, version, _runtime_, deploy settings, metadata ...
* a `requirements.txt` package dependencies file
* a `main.py` webscript script
* additional scripts we added above, like the `autoencoder.py` and the `model-weights.pt`

In [31]:
## configure any 'memory' or 'cpu' deploy settings 
deploy_overrides = {'limits' : { 'memory': '2G' }, 'requests' : { 'memory' : '1G' }}
## configure the webscript to use our model
adapter = adapter.as_plug({
    'name': MODEL_NAME, 
    'description':'pytorch autoencoder for caats', 
    'deploy' : deploy_overrides                                                                                          
})

IndexError: Multiple assets of type <class 'ml_adapter.base.assets.manifest.FunctionManifestAsset'> found: [webscript.json <ml_adapter.base.assets.manifest.WebscriptManifestAsset>, plug.json <ml_adapter.base.assets.manifest.PlugManifestAsset>]

In [34]:
del adapter.assets.children[1]

In [36]:
adapter.assets.children

[openapi.json <ml_adapter.base.assets.openapi.OpenApiAsset>,
 requirements.txt <ml_adapter.base.assets.python.PythonRequirementsAsset>,
 main.py <ml_adapter.base.assets.python.PythonScriptAsset>,
 lib <ml_adapter.base.assets.python.PythonLibAssetDir>,
 model-weights.pt <ml_adapter.torch.adapter.TorchModelWeightsAsset>,
 plug.json <ml_adapter.base.assets.manifest.PlugManifestAsset>,
 autoencoder.py <ml_adapter.base.assets.python.PythonScriptAsset>]

In [20]:
await adapter.save()
list(a.path for a in adapter.assets)

['openapi.json',
 'requirements.txt',
 'main.py',
 'model-weights.pt',
 'autoencoder.py',
 'plug.json']

In [21]:
# lets have a look at the generated python plug:
display(Code(filename=f'{ARCHIVE_LOC}/main.py'))

# You could adapt this script to have specific error handling or handling of request/response

# ml_adapter.torch.adapter.V1TorchAdapter model adapter
import os
from ml_adapter.api.data import v1 as V1
from ml_adapter.torch.adapter import V1TorchAdapter

# optional type alias for plug response
StatusAndRawData = tuple[str, V1.V1PredictionResponse|V1.V1ErrorResponse]

STATE_OK = 'PREDICTED'
STATE_NOK = 'FAILED'

MODEL_PATH = os.environ.get('MODEL_PATH', 'model-weights.pt')
MODEL_CLASS = os.environ.get('MODEL_CLASS', 'autoencoder.AutoEncoder')

# Initialize the model adapter.
# Provide a `model` argument if you want to create/load the model yourself.
adapter = V1TorchAdapter(
    model_path=MODEL_PATH, model_class=MODEL_CLASS
)

async def execute(properties: V1.V1Request, console, logger) -> StatusAndRawData:
    try:
        result = await adapter.call(properties)
        return (STATE_OK, result)
    except Exception as err:
        logger.exception(err)
        error_message = str(err)
        console.error(error_message)
        return (STATE_NOK, { 'error': error_message, 'predictions': [] })

In [22]:
# once stored, the adapter can be restored later with
# adapter = await V1TorchAdapter(model_path='model-weights.pt', model_class=AutoEncoder, location=ARCHIVE_LOC).load()

In [37]:
list(adapter.assets)

[openapi.json <ml_adapter.base.assets.openapi.OpenApiAsset>,
 requirements.txt <ml_adapter.base.assets.python.PythonRequirementsAsset>,
 main.py <ml_adapter.base.assets.python.PythonScriptAsset>,
 model-weights.pt <ml_adapter.torch.adapter.TorchModelWeightsAsset>,
 plug.json <ml_adapter.base.assets.manifest.PlugManifestAsset>,
 autoencoder.py <ml_adapter.base.assets.python.PythonScriptAsset>]

In [38]:
list(a.path for a in adapter.assets)

['openapi.json',
 'requirements.txt',
 'main.py',
 'model-weights.pt',
 'plug.json',
 'autoencoder.py']

### Uploading the plug using the SDK
To upload these assets and create a plug, we need to call the [create plug](https://docs.waylay.io/openapi/public/redocly/registry.html#tag/Plugs/operation/create_plugs) REST api. 

The code belows uses the `ml_tool` plugin to handle this.
Alternatively you could call `await adapter.save_archive()`
which creates an `autoencoder-pytorch.tar.gz` archive that you can upload as a plug with `curl` or using the waylay console.

In [39]:
from waylay.sdk import WaylayClient

In [40]:
# check the SDK client
client = WaylayClient.from_profile('staging')

In [41]:
ref = await client.ml_tool.create_plug(adapter)
ref

2024-06-11 14:53:22 INFO     HTTP Request: POST https://api-aws-dev.waylay.io/accounts/v1/tokens?grant_type=client_credentials "HTTP/1.1 200 OK"
2024-06-11 14:53:23 INFO     HTTP Request: POST https://api-aws-dev.waylay.io/registry/v2/plugs/?draft=false&comment=&async=true "HTTP/1.1 202 Accepted"


{'message': 'Building and deploying plug autoencoderV1@0.0.1',
 '_links': {'event': {'href': 'https://api-aws-dev.waylay.io/registry/v2/jobs/events?type=verify&id=740799ef-d515-4704-8718-903851c9899e$SxMJXAcOF0Aqs9UljAFR8&children=true'},
  'job': {'href': 'https://api-aws-dev.waylay.io/registry/v2/jobs/verify/740799ef-d515-4704-8718-903851c9899e$SxMJXAcOF0Aqs9UljAFR8'}},
 'entity': {'createdBy': 'users/edb8841f-122e-4f7d-a412-397764bc9996',
  'createdAt': '2024-06-11T12:53:23.376Z',
  'updatedBy': 'users/edb8841f-122e-4f7d-a412-397764bc9996',
  'updatedAt': '2024-06-11T12:53:23.400Z',
  'updates': [{'operation': 'create',
    'at': '2024-06-11T12:53:23.400Z',
    'by': 'users/edb8841f-122e-4f7d-a412-397764bc9996',
    'comment': '',
    'jobs': ['740799ef-d515-4704-8718-903851c9899e$SxMJXAcOF0Aqs9UljAFR8',
     '740799ef-d515-4704-8718-903851c9899e$OSFfqJLE9kXXLNUv45koB',
     '740799ef-d515-4704-8718-903851c9899e$UypvDQdH-_nv6HBnogjWM']}],
  'status': 'pending',
  'runtime': {'deprec

In [ ]:
# wait until the build, deploy and verify jobs for the webscript have finished
# NOTE: this might take quite a few minutes, as the dependencies for torch webscripts are quite big and the images not yet optimised
ref = await client.ml_tool.wait_until_ready(ref)

In [ ]:
# test the webscript invocation
await client.ml_tool.test_webscript(ref, x_data.tolist())

In [ ]:
# remove the webscript
await client.ml_tool.remove(ref)

#### About `ml_tool`
The `client.ml_tool` methods are essentialy wrappers around the methods of the [registry](https://docs.waylay.io/openapi/public/redocly/registry.html#tag/Webscripts) service. Alternatively you can use the [`client.registry.webscript`](https://github.com/waylayio/waylay-sdk-registry-py) methods directly.